# Distributed HPO with Ray Tune and XGBoost-Ray

This demo introduces **Ray tune's** key concepts using a a classification example. This example is derived from [Hyperparametere Tuning with Ray Tune and XGBoost-Ray](https://github.com/ray-project/xgboost_ray#hyperparameter-tuning). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#blendsearch) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

import ray
from ray import tune
CONNECT_TO_ANYSCALE=True

In [2]:
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-weekly-demo")
    else:
        ray.init()

Authenticating
Loaded Anyscale authentication token from ANYSCALE_CLI_TOKEN.

Output
(anyscale +2.6s) WARNING: No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
(anyscale +3.6s) .anyscale.yaml found in project_dir. Directory is attached to a project.
(anyscale +10.0s) Using project (name: prj-weekly-demo, project_dir: /Users/jules/git-repos/ray-core-tutorial, id: prj_5rvR1w2ciyUs9RM27FeZ6FVB).
(anyscale +28.6s) cluster jsd-weekly-demo is currently running, the cluster will not be restarted.


2022-01-05 16:11:51,786	INFO packaging.py:352 -- Creating a file package for local directory '/Users/jules/git-repos/ray-core-tutorial'.
2022-01-05 16:11:51,808	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_6b65e788e5623db5.zip' (1.91MiB) to Ray cluster...
2022-01-05 16:11:55,030	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_6b65e788e5623db5.zip'.


(anyscale +54.5s) Connected to jsd-weekly-demo, see: https://console.anyscale.com/projects/prj_5rvR1w2ciyUs9RM27FeZ6FVB/clusters/ses_GKnU1MGrXqZNmAkicHxSKU9G
(anyscale +54.5s) URL for head node of cluster: https://session-gknu1mgrxqznmakichxsku9g.i.anyscaleuserdata.com


## Step 1: Define a 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [4]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

In [5]:
ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [6]:
def train_func_model(config:dict, checkpoint_dir=None):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define a hyperparameter search space

In [7]:
 # Specify the hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="50%" width="60%"> 

In [8]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources()
)

(run pid=15901) == Status ==
(run pid=15901) Current time: 2022-01-05 16:12:31 (running for 00:00:00.12)
(run pid=15901) Memory usage on this node: 3.0/62.0 GiB
(run pid=15901) Using FIFO scheduling algorithm.
(run pid=15901) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.2 GiB heap, 0.0/92.17 GiB objects
(run pid=15901) Result logdir: /home/ray/ray_results/train_func_model_2022-01-05_16-12-30
(run pid=15901) Number of trials: 4/4 (4 PENDING)
(run pid=15901) +------------------------------+----------+-------+-------------+-------------+-------------+
(run pid=15901) | Trial name                   | status   | loc   |         eta |   max_depth |   subsample |
(run pid=15901) |------------------------------+----------+-------+-------------+-------------+-------------|
(run pid=15901) | train_func_model_56b65_00000 | PENDING  |       | 0.000421166 |           1 |    0.857729 |
(run pid=15901) | train_func_model_56b65_00001 | PENDING  |       | 0.00333829  |           1 |    0.85758  |


(ImplicitFunc pid=1361, ip=172.31.74.136) 2022-01-05 16:12:33,849	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=1361, ip=172.31.85.248) 2022-01-05 16:12:33,870	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=16040) 2022-01-05 16:12:33,858	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=1389, ip=172.31.82.103) 2022-01-05 16:12:33,992	INFO main.py:926 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=1361, ip=172.31.74.136) 2022-01-05 16:12:35,967	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(ImplicitFunc pid=16040) 2022-01-05 16:12:36,078	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=1395, ip=172.31.74.136

(run pid=15901) Result for train_func_model_56b65_00001:
(run pid=15901)   date: 2022-01-05_16-12-36
(run pid=15901)   done: false
(run pid=15901)   experiment_id: df29a36ace8f43959d8b9c9fa0a1b612
(run pid=15901)   hostname: ip-172-31-74-136
(run pid=15901)   iterations_since_restore: 1
(run pid=15901)   node_ip: 172.31.74.136
(run pid=15901)   pid: 1361
(run pid=15901)   time_since_restore: 3.2334842681884766
(run pid=15901)   time_this_iter_s: 3.2334842681884766
(run pid=15901)   time_total_s: 3.2334842681884766
(run pid=15901)   timestamp: 1641427956
(run pid=15901)   timesteps_since_restore: 0
(run pid=15901)   train-error: 0.079086
(run pid=15901)   train-logloss: 0.690847
(run pid=15901)   training_iteration: 1
(run pid=15901)   trial_id: 56b65_00001
(run pid=15901)   
(run pid=15901) 2022-01-05 16:12:37,030	WARN commands.py:269 -- Loaded cached provider configuration
(run pid=15901) 2022-01-05 16:12:37,030	WARN commands.py:270 -- If you experience issues with the cloud provider,

(ImplicitFunc pid=1361, ip=172.31.85.248) 2022-01-05 16:13:02,764	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 28.93 seconds (26.67 pure XGBoost training time).
(ImplicitFunc pid=1361, ip=172.31.74.136) 2022-01-05 16:13:02,769	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 28.96 seconds (26.80 pure XGBoost training time).
(ImplicitFunc pid=1389, ip=172.31.82.103) 2022-01-05 16:13:02,765	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 28.81 seconds (26.55 pure XGBoost training time).
(ImplicitFunc pid=16040) 2022-01-05 16:13:02,779	INFO main.py:1450 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 28.95 seconds (26.70 pure XGBoost training time).


(run pid=15901) == Status ==
(run pid=15901) Current time: 2022-01-05 16:13:02 (running for 00:00:31.25)
(run pid=15901) Memory usage on this node: 3.8/62.0 GiB
(run pid=15901) Using FIFO scheduling algorithm.
(run pid=15901) Resources requested: 20.0/80 CPUs, 0/0 GPUs, 0.0/216.2 GiB heap, 0.0/92.17 GiB objects
(run pid=15901) Current best trial: 56b65_00003 with train-error=0.02109 and parameters={'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0001681580711826371, 'subsample': 0.9610347499653313, 'max_depth': 6, 'nthread': 1, 'n_jobs': 1}
(run pid=15901) Result logdir: /home/ray/ray_results/train_func_model_2022-01-05_16-12-30
(run pid=15901) Number of trials: 4/4 (4 RUNNING)
(run pid=15901) +------------------------------+----------+---------------------+-------------+-------------+-------------+--------+------------------+-----------------+---------------+
(run pid=15901) | Trial name                   | status   | loc         

(run pid=15901) 2022-01-05 16:13:02,905	INFO tune.py:626 -- Total run time: 32.30 seconds (31.35 seconds for the tuning loop).


In [9]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0001681580711826371, 'subsample': 0.9610347499653313, 'max_depth': 6}


In [10]:
analysis.results_df.head(5)

/usr/local/anaconda3/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,iterations_since_restore,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
56b65_00000,0.690224,0.065026,0.018358,True,None,None,10,cff0a76fef0744e8b43791f357b70b51,2022-01-05_16-13-02,1641427982,...,10,"0_eta=0.00042117,max_depth=1,subsample=0.85773",approx,binary:logistic,"[logloss, error]",0.000421,0.857729,1,1,1
56b65_00001,0.670509,0.065026,0.005539,True,None,None,10,df29a36ace8f43959d8b9c9fa0a1b612,2022-01-05_16-13-02,1641427982,...,10,"1_eta=0.0033383,max_depth=1,subsample=0.85758",approx,binary:logistic,"[logloss, error]",0.003338,0.857580,1,1,1
56b65_00002,0.619979,0.012302,0.007786,True,None,None,10,b4093145b82d4dea899e866c3aae0a31,2022-01-05_16-13-02,1641427982,...,10,"2_eta=0.0091453,max_depth=7,subsample=0.77319",approx,binary:logistic,"[logloss, error]",0.009145,0.773193,7,1,1
56b65_00003,0.691659,0.008787,0.005809,True,None,None,10,ce2b3dd01ec447d9aa745ecf99ec0338,2022-01-05_16-13-02,1641427982,...,10,"3_eta=0.00016816,max_depth=6,subsample=0.96103",approx,binary:logistic,"[logloss, error]",0.000168,0.961035,6,1,1


In [11]:
ray.shutdown()

---

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](https://docs.ray.io/en/latest/xgboost-ray.html)